[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-1/deployment.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239303-lesson-8-deployment)

# Развертывание

## Обзор

Мы создали агента с памятью:

* `act` - позволить модели вызывать конкретные инструменты
* `observe` - передавать вывод инструмента обратно в модель
* `reason` - позволить модели размышлять о выводах инструмента, чтобы решить, что делать дальше (например, вызвать другой инструмент или просто ответить напрямую)
* `persist state` - использовать контрольную точку в памяти для поддержки длительных разговоров с перерывами

## Цели

Теперь мы рассмотрим, как развернуть нашего агента локально в Studio и в `LangGraph Cloud`.

In [3]:
%%capture --no-stderr
%pip install --quiet -U langgraph_sdk langchain_core

## Концепции

Есть несколько центральных концепций, которые нужно понять —

`LangGraph` —
- Библиотека для Python и JavaScript
- Позволяет создавать рабочие процессы агентов

`LangGraph API` —
- Упаковывает код графа
- Предоставляет очередь задач для управления асинхронными операциями
- Обеспечивает постоянство для поддержания состояния между взаимодействиями

`LangGraph Cloud` —
- Хостинговая служба для LangGraph API
- Позволяет развертывать графы из репозиториев GitHub
- Также предоставляет мониторинг и трассировку для развернутых графов
- Доступен через уникальный URL для каждого развертывания

`LangGraph Studio` —
- Интегрированная среда разработки (IDE) для приложений LangGraph
- Использует API как серверную часть, позволяя тестировать и исследовать графы в реальном времени
- Может быть запущен локально или с развертыванием в облаке

`LangGraph SDK` —
- Библиотека Python для программного взаимодействия с графами LangGraph
- Предоставляет согласованный интерфейс для работы с графами, независимо от того, обслуживаются ли они локально или в облаке
- Позволяет создавать клиентов, получать доступ к ассистентам, управлять потоками и выполнять запуска

## Тестирование локально

--

**⚠️ ОТКАЗ ОТ ОТВЕТСТВЕННОСТИ**

*Для работы Studio требуется Mac. Если вы не используете Mac, пропустите этот шаг.*

*Если вы работаете с этим ноутбуком в CoLab, пропустите этот шаг.*

--

Мы можем легко подключиться к графам, которые обслуживаются локально в LangGraph Studio!

Мы делаем это через `url`, предоставленный в нижнем левом углу интерфейса Studio.

![Screenshot 2024-08-23 at 1.17.05 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4f53080e6802cec34d_deployment%201.png)

In [1]:
import platform

if 'google.colab' in str(get_ipython()) or platform.system() != 'Darwin':
    raise Exception("Unfortunately LangGraph Studio is currently not supported on Google Colab or requires a Mac")

In [2]:
from langgraph_sdk import get_client

In [5]:
# Replace this with the URL of your own deployed graph
URL = "http://localhost:64157"
client = get_client(url=URL)

# Search all hosted graphs
assistants = await client.assistants.search()

In [6]:
assistants[-3]

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'created_at': '2024-12-28T07:01:53.358131+00:00',
 'updated_at': '2024-12-28T07:01:53.358131+00:00',
 'config': {},
 'metadata': {'created_by': 'system'},
 'version': 1,
 'name': 'agent'}

In [7]:
# Мы создаем поток для отслеживания состояния нашего запуска
thread = await client.threads.create()

Теперь мы можем запустить нашего агента с помощью [client.runs.stream](https://langchain-ai.github.io/langgraph/concepts/low_level/#stream-and-astream) с:

* `thread_id`
* `graph_id`
* `input`
* `stream_mode`

Мы подробно рассмотрим стриминг в следующем модуле.

Пока просто примите к сведению, что мы [стримим](https://langchain-ai.github.io/langgraph/cloud/how-tos/stream_values/) полное значение состояния после каждого шага графа с `stream_mode="values"`.

Состояние захватывается в `chunk.data`.

In [8]:
from langchain_core.messages import HumanMessage

# Input
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# Stream
async for chunk in client.runs.stream(
        thread['thread_id'],
        "agent",
        input=input,
        stream_mode="values",
    ):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data['messages'][-1])

{'content': 'Multiply 3 by 2.', 'additional_kwargs': {'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'e63656e6-09d1-456f-9c8b-9d597d728b3b', 'example': False}
{'content': '', 'additional_kwargs': {'tool_calls': [{'id': 'call_droXbSErsfqREGWVixry3l4S', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 18, 'prompt_tokens': 135, 'total_tokens': 153, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_5f20662549', 'finish_reason': 'tool_calls', 'logprobs': None}, 'type': 'ai', 'name': None, 'id': 'run-d8da96d3-96c1-4754-9194-a81b1ac16017-0', 'example': False, 'tool_calls': [{'name': 'm

## Тестирование в облаке

Мы можем развернуть в Cloud через LangSmith, как описано [здесь](https://langchain-ai.github.io/langgraph/cloud/quick_start/#deploy-from-github-with-langgraph-cloud).

### Создайте новый репозиторий на GitHub

* Перейдите в ваш аккаунт на GitHub
* Нажмите на иконку "+" в правом верхнем углу и выберите `"New repository"`
* Назовите ваш репозиторий (например, `langchain-academy`)

### Добавьте ваш репозиторий GitHub как удаленный

* Вернитесь в терминал, где вы клонировали `langchain-academy` в начале этого курса
* Добавьте ваш только что созданный репозиторий GitHub как удаленный

###
git remote add origin https://github.com/your-username/your-repo-name.git
###
* Запушьте изменения
###
git push -u origin main
###

### Подключите LangSmith к вашему репозиторию GitHub

* Перейдите на [LangSmith](hhttps://smith.langchain.com/)
* Нажмите на вкладку `deployments` в левом меню LangSmith
* Добавьте `+ New Deployment`
* Затем выберите GitHub репозиторий (например, `langchain-academy`), который вы только что создали для курса
* Укажите `LangGraph API config file` на одну из директорий `studio`
* Например, для модуля-1 используйте: `module-1/studio/langgraph.json`
* Установите ваши API ключи (например, можно просто скопировать из файла `module-1/studio/.env`)

### Работайте с вашим развертыванием

Мы можем взаимодействовать с нашим развертыванием несколькими способами:

* С помощью [SDK](https://langchain-ai.github.io/langgraph/cloud/quick_start/#use-with-the-sdk), как и раньше.
* С помощью [LangGraph Studio](https://langchain-ai.github.io/langgraph/cloud/quick_start/#interact-with-your-deployment-via-langgraph-studio).

![Screenshot 2024-08-23 at 10.59.36 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4fa159a09a51d601de_deployment3.png)

Чтобы использовать SDK здесь, в ноутбуке, убедитесь, что `LANGSMITH_API_KEY` установлен!

In [1]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("LANGCHAIN_API_KEY")

In [ ]:
# Replace this with the URL of your deployed graph
URL = "https://langchain-academy-8011c561878d50b1883f7ed11b32d720.default.us.langgraph.app"
client = get_client(url=URL)

# Search all hosted graphs
assistants = await client.assistants.search()

In [37]:
# Select the agent
agent = assistants[0]

In [38]:
agent

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'created_at': '2024-08-23T17:58:02.722920+00:00',
 'updated_at': '2024-08-23T17:58:02.722920+00:00',
 'config': {},
 'metadata': {'created_by': 'system'}}

In [40]:
from langchain_core.messages import HumanMessage

# We create a thread for tracking the state of our run
thread = await client.threads.create()

# Input
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# Stream
async for chunk in client.runs.stream(
        thread['thread_id'],
        "agent",
        input=input,
        stream_mode="values",
    ):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data['messages'][-1])

{'content': 'Multiply 3 by 2.', 'additional_kwargs': {'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '8ea04559-f7d4-4c82-89d9-c60fb0502f21', 'example': False}
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_EQoolxFaaSVU8HrTnCmffLk7', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27'}, 'type': 'ai', 'name': None, 'id': 'run-b0ea5ddd-e9ba-4242-bb8c-80eb52466c76', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_EQoolxFaaSVU8HrTnCmffLk7', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': None}
{'content': '6', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'multiply', 'id': '1bf558e7-79ef-4f21-bb66-acafbd04677a', 'tool_call_id': 'c